In [33]:
import torch
from OFA_mbv3_extended.networks.nets.my_networks import get_net_by_name, get_teacher_by_name
from OFA_mbv3_extended.utils.pytorch_utils import get_net_info
from fvcore.nn import FlopCountAnalysis
from torchprofile import profile_macs
import warnings

In [22]:
args_st = {
    "n_classes": 200,
    "dropout_rate":0.2,
    "width_mult":1.0,
    "ks":7,
    "expand_ratio":6,
    "depth_param": 4
}

In [28]:
mbv3 = get_teacher_by_name("OFAMobileNetV3")(**args_st)
seb = get_teacher_by_name("SE_B_OFAMobileNetV3")(**args_st)
sed = get_teacher_by_name("SE_D_OFAMobileNetV3")(**args_st)
sep = get_teacher_by_name("SE_P_OFAMobileNetV3")(**args_st)
sedp = get_teacher_by_name("SE_DP_OFAMobileNetV3")(**args_st)
eeb = get_teacher_by_name("EE_B_OFAMobileNetV3")(**args_st)
eed = get_teacher_by_name("EE_D_OFAMobileNetV3")(**args_st)
eep = get_teacher_by_name("EE_P_OFAMobileNetV3")(**args_st)
eedp = get_teacher_by_name("EE_DP_OFAMobileNetV3")(**args_st)
st_nets = [mbv3, seb, sed, sep, sedp, eeb, eed, eep, eedp]

In [29]:
input_size = (1,3,64,64)

In [30]:
dummy_data = torch.rand(*input_size)

# OFA

In [34]:
for net in st_nets:
    flops, _ = get_net_info(net, (3,64, 64))
    print(flops)

latency types:  []
Total training params: 6.64M
Total FLOPs: 51.13M
params
latency types:  []
Total training params: 6.64M
Total FLOPs: 51.13M
params
latency types:  []
Total training params: 6.64M
Total FLOPs: 51.13M
params
latency types:  []
Total training params: 7.34M
Total FLOPs: 90.28M
params
latency types:  []
Total training params: 7.34M
Total FLOPs: 90.28M
params
latency types:  []
Total training params: 8.19M
Total FLOPs: 56.16M
params
latency types:  []
Total training params: 8.19M
Total FLOPs: 56.16M
params
latency types:  []
Total training params: 8.90M
Total FLOPs: 95.31M
params
latency types:  []
Total training params: 8.90M
Total FLOPs: 95.31M
params


# MACS

In [15]:
def _calc_flops(subnet, dummy_data):
    return profile_macs(subnet, dummy_data) / 1e6  # in unit of MFLOPs

In [16]:
for net in st_nets:
    with warnings.catch_warnings():  # ignore warnings, use w/ caution
        warnings.simplefilter("ignore")
        flops=_calc_flops(net, dummy_data)
        print(flops)

50.910976
50.910976
50.910976
94.033152
94.033152
55.805824
55.805824
98.928
98.928


# ==> dense connections ignored

# FACEBOOK

In [18]:
def fvflops(subnet, dummy_data):
    fca = FlopCountAnalysis(subnet, dummy_data)
    flops= fca.total()
    return flops/1e6

In [19]:
for net in st_nets:
    with warnings.catch_warnings():  # ignore warnings, use w/ caution
        warnings.simplefilter("ignore")
        flops=fvflops(net, dummy_data)
        print(flops)

Unsupported operator aten::add_ encountered 64 time(s)
Unsupported operator aten::add encountered 55 time(s)
Unsupported operator aten::relu6_ encountered 38 time(s)
Unsupported operator aten::mul encountered 39 time(s)
Unsupported operator aten::div encountered 39 time(s)
Unsupported operator aten::mean encountered 26 time(s)
Unsupported operator aten::relu6 encountered 1 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
blocks.0.shortcut, blocks.10.shortcut, blocks.11.shortcut, blocks.12.shortcut, blocks.14.shortcut, blocks.15.shortcut, blocks.16.shortcut, blocks.18.shortcut, blocks.19.shortcut, blocks.2.shortcut, blocks.20.shortcut, blocks.3.shortcut,

53.630976


Unsupported operator aten::add_ encountered 64 time(s)
Unsupported operator aten::add encountered 55 time(s)
Unsupported operator aten::relu6_ encountered 38 time(s)
Unsupported operator aten::mul encountered 39 time(s)
Unsupported operator aten::div encountered 39 time(s)
Unsupported operator aten::mean encountered 26 time(s)
Unsupported operator aten::relu6 encountered 1 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
body_stages.0.layers.1.shortcut, body_stages.0.layers.2.shortcut, body_stages.0.layers.3.shortcut, body_stages.1.layers.1.shortcut, body_stages.1.layers.2.shortcut, body_stages.1.layers.3.shortcut, body_stages.2.layers.1.shortcut, body_

53.630976


Unsupported operator aten::add_ encountered 79 time(s)
Unsupported operator aten::add encountered 55 time(s)
Unsupported operator aten::relu6_ encountered 38 time(s)
Unsupported operator aten::mul encountered 39 time(s)
Unsupported operator aten::div encountered 39 time(s)
Unsupported operator aten::mean encountered 26 time(s)
Unsupported operator aten::relu6 encountered 1 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
body_stages.0.layers.1.shortcut, body_stages.0.layers.2.shortcut, body_stages.0.layers.3.shortcut, body_stages.1.layers.1.shortcut, body_stages.1.layers.2.shortcut, body_stages.1.layers.3.shortcut, body_stages.2.layers.1.shortcut, body_

...

58.725376


Unsupported operator aten::add_ encountered 128 time(s)
Unsupported operator aten::add encountered 135 time(s)
Unsupported operator aten::relu6_ encountered 54 time(s)
Unsupported operator aten::mul encountered 79 time(s)
Unsupported operator aten::div encountered 59 time(s)
Unsupported operator aten::unfold encountered 80 time(s)
Unsupported operator aten::softmax encountered 20 time(s)
Unsupported operator aten::avg_pool2d encountered 4 time(s)
Unsupported operator aten::mean encountered 34 time(s)
Unsupported operator aten::relu6 encountered 5 time(s)
Unsupported operator aten::dropout_ encountered 5 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
body_stages.0.layers_blocks.1.0.shortcut, body_stages.0.lay

102.020702


Unsupported operator aten::add_ encountered 143 time(s)
Unsupported operator aten::add encountered 135 time(s)
Unsupported operator aten::relu6_ encountered 54 time(s)
Unsupported operator aten::mul encountered 79 time(s)
Unsupported operator aten::div encountered 59 time(s)
Unsupported operator aten::unfold encountered 80 time(s)
Unsupported operator aten::softmax encountered 20 time(s)
Unsupported operator aten::avg_pool2d encountered 4 time(s)
Unsupported operator aten::mean encountered 34 time(s)
Unsupported operator aten::relu6 encountered 5 time(s)
Unsupported operator aten::dropout_ encountered 5 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
body_stages.0.layers_blocks.1.0.shortcut, body_stages.0.lay

102.020702
